In [5]:
import tensorflow as tf

from azureml.core import Run
from azureml.core import Experiment, Workspace

In [6]:
ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='textgen-lstm')

run = Run(exp, 'textgen-lstm_1611696706_5b26c855')

In [7]:
# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/'):
        print(f.replace('outputs/model/', ""))
        output_path = os.path.join('./model/', f.replace('outputs/model/', ""))
        print('Downloading from {} to {} ...'.format(f, output_path))
        run.download_file(name=f, output_file_path=output_path)

model/saved_model.pb
model/variables/variables.data-00000-of-00001
model/variables/variables.index
one_step_model/saved_model.pb
one_step_model/variables/variables.data-00000-of-00001
one_step_model/variables/variables.index
outputs/training_checkpoints/checkpoint
outputs/training_checkpoints/ckpt_1.data-00000-of-00001
outputs/training_checkpoints/ckpt_1.index
outputs/training_checkpoints/ckpt_1_temp_ffff69fb13fc40c180bf78806e516bc1/part-00000-of-00001.index


Failed to download file with error: The specified blob does not exist. ErrorCode: BlobNotFound
<?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:7c5e9fb1-e01e-0039-4b9d-f455a3000000
Time:2021-01-27T11:16:27.5321604Z</Message></Error>


AzureMLException: AzureMLException:
	Message: Download of file failed with error: The specified blob does not exist. ErrorCode: BlobNotFound
<?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:7c5e9fb1-e01e-0039-4b9d-f455a3000000
Time:2021-01-27T11:16:27.5321604Z</Message></Error>
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Download of file failed with error: The specified blob does not exist. ErrorCode: BlobNotFound\n<?xml version=\"1.0\" encoding=\"utf-8\"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.\nRequestId:7c5e9fb1-e01e-0039-4b9d-f455a3000000\nTime:2021-01-27T11:16:27.5321604Z</Message></Error>"
    }
}

In [8]:
one_step_reloaded = tf.saved_model.load('model/one_step_model')

In [3]:




states = None
next_char = tf.constant(['pan to jednak'])
result = [x+' ' for x in next_char]

for n in range(6):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char+' ')

result = tf.strings.join(result)

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

NameError: name 'one_step_reloaded' is not defined

In [7]:
model = run.register_model(model_name='textgen_model', model_path='outputs/model/')

In [8]:
model

Model(workspace=Workspace.create(name='sentencecompletion', subscription_id='5334814e-153c-45f0-9557-00aa820611b9', resource_group='SentenceCompletionAzure'), name=textgen_model, id=textgen_model:1, version=1, tags={}, properties={})

In [4]:
%%writefile gen.py
import json
import numpy as np
import os
import tensorflow as tf

from azureml.core.model import Model

def init():
    global model
    model_root = os.getenv('AZUREML_MODEL_DIR')
    model_folder = 'model/one_step_model'
    model = tf.saved_model.load(os.path.join(model_root, model_folder))
 
    
def run(input_):
    json_ = json.loads(input_)
    next_word = tf.constant([json_["data"]])
    n = json_['num']

    json_1 = { "data" : [] }
    for i in range(n):
        states = None
        next_word = tf.constant(['pan to jednak'])
        result = [x+' ' for x in next_word]

        for n in range(6):
            next_word, states = model.generate_one_step(next_word, states=states)
            result.append(next_word+' ')

        json_1["data"].append(tf.strings.join(result)[0].numpy().decode("utf-8").replace('\r\n', " "))

    return json_1



Writing gen.py


In [9]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_pip_package('tensorflow==2.4.1')
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - tensorflow==2.4.1
  - azureml-defaults
- numpy
channels:
- anaconda
- conda-forge



In [10]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="gen.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=3.8,
                                               auth_enabled=True, 
                                               memory_gb=15,
                                               tags={'name': 'textgen', 'framework': 'TfKeras'},
                                               description='text-generation')

service = Model.deploy(workspace=ws, 
                           name='text-generation-c', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [11]:
print(service.get_logs())

2021-01-27T00:32:25,878851108+00:00 - gunicorn/run 
2021-01-27T00:32:25,880882594+00:00 - rsyslog/run 
2021-01-27T00:32:25,886901753+00:00 - iot-server/run 
2021-01-27T00:32:25,889988831+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f0880b375e57ffe758b94a0c72756a7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC